In [2]:
import matlab
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'C:/Users/igorp/OneDrive/Documents/GitHub/Project-Heart/matlab',nargout=0)

In [3]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV
from project_heart.enums import *

In [153]:
dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
files = sorted(os.listdir(dir))

file_id = 1 # 5 has a problem

print("File:", files[file_id])

old_lv_file = "C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt"

lv = LV()
lv.from_pyvista_read(dir/files[file_id], identifier="elemTag", threshold=[0, 1])

File: Full_Heart_Mesh_10.vtk


In [155]:
import tetgen
lv_surf = lv.get_surface_mesh().triangulate().smooth(100)
tet = tetgen.TetGen(lv_surf)
tet.make_manifold()
tet.tetrahedralize(order=1, 
                   nobisect=True,
                  #  quality=False,
                   mindihedral=100, 
                   minratio=1.1,
                  #  steinerleft=-1
                   )
lv_tet = LV()
lv_tet.from_pyvista_dataset(tet.grid)
lv_tet

In [156]:
lv = lv_tet

In [157]:
lv.identify_base_and_apex_regions()

(array([6., 6., 6., ..., 6., 0., 0.]), array([6., 6., 6., ..., 0., 0., 0.]))

In [158]:
lv.identify_surfaces(
  
  ab_ql=0.03, 
  ab_qh=0.70,
  
  alpha_atr=0.20,  # coeff for radial distance computation
  alpha_mtr=0.69,
  
  beta_atr=0.15,  # coeff for second radial distance computation
  beta_mtr=0.27,
  
  gamma_atr=89,
  gamma2_mtr=25,
  
  phi_atr=79,
  epi_angle=100
)

(array([6., 6., 6., ..., 6., 0., 0.]), array([6., 6., 6., ..., 0., 0., 0.]))

In [164]:
poi = set(np.where(lv.mesh.point_data["LV_SURFS"] == LV_SURFS.ENDO)[0])

In [165]:
# poi_1 = set(lv.get_nodeset(LV_SURFS.MITRAL.name))
# poi_2 = set(lv.get_nodeset(LV_SURFS.AORTIC.name))
# poi_3 = set(lv.get_nodeset(LV_SURFS.AM_INTERCECTION.name))
# poi_4 = set(lv.get_nodeset(LV_SURFS.EPI.name))



cells = lv.cells()[VTK_ELEMENTS.TETRA.value]
regionIds = np.ones(len(cells), dtype=np.int64)
cells_flat = cells.flatten()
for idx, cell in enumerate(cells_flat):
  if cell in poi:
    regionIds[int(np.floor(idx/4))] = 2
  # if cell in poi_2:
  #   regionIds[int(np.floor(idx/4))] = 3
  # if cell in poi_3:
  #   regionIds[int(np.floor(idx/4))] = 4

In [168]:
edges = lv.get_surface_mesh().extract_feature_edges(feature_angle=20, non_manifold_edges=False).extract_largest()
eoi = edges.extract_largest().point_data["vtkOriginalPointIds"]
eoi = set(eoi)

cells = lv.cells()[VTK_ELEMENTS.TETRA.value]
regionIds = np.ones(len(cells), dtype=np.int64)
cells_flat = cells.flatten()
for idx, cell in enumerate(cells_flat):
  if cell in eoi:
    regionIds[int(np.floor(idx/4))] = 2

In [169]:
lv.mesh.cell_data["rids"] = regionIds
lv.mesh.set_active_scalars("rids")
lv.mesh.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [170]:
tet_nodes = lv.points()
tet_cells = lv.cells(as_json_ready=True)["TETRA"]+1

np.savetxt("nodes.csv", tet_nodes, delimiter=",")
np.savetxt("elems.csv", tet_cells.astype(np.int32), fmt='%i', delimiter=",")
np.savetxt("regionIDs.csv", regionIds, fmt='%i', delimiter=",")